# Misc. Astronomy Tools

In [6]:
%%javascript

document.head.insertAdjacentHTML('beforeend', '<style>.container { width:99% !important; }</style>');

<IPython.core.display.Javascript object>

### Stars around a coordinate

In [44]:
# Stars aroudn a coordiante, from GAIA DR2/EDR3
from astropy.coordinates import SkyCoord, Angle
from astroquery.vizier import Vizier

def move_column(df, column_name, insert_position):
    """Move the named column to the posistion in a pandas dataframe"""
    col = df.pop(column_name)
    df.insert(insert_position, col.name, col)
    return df

def search_nearby(ra, dec, catalog_name="I/345/gaia2", radius_arcsec=30, magnitude_limit=21):
    
    c1 = SkyCoord(ra, dec, frame="icrs", unit="deg")
    Vizier.ROW_LIMIT = -1
    columns = ["*"]
    if catalog_name == "I/350/gaiaedr3":
        columns = ["*", "epsi", "sepsi"]  # add astrometric excess noise to the output (see if a star wobbles)

    result = Vizier(columns=columns).query_region(
        c1,
        catalog=[catalog_name],
        radius=Angle(radius_arcsec, "arcsec"),
    )
    if len(result) < 1:
        return None
    result = result[catalog_name]
#     result = result.to_pandas()
#     if result.get("Gmag") is not None:
#         result = result[result.Gmag < magnitude_limit]
#     if result.get("RUWE") is not None:
#         move_column(result, "RUWE", 7)        
#     if result.get("sepsi") is not None:
#         move_column(result, "sepsi", 8)        
#     if result.get("epsi") is not None:
#         move_column(result, "epsi", 9)        
    return result


catalog_name = "I/345/gaia2"      # https://vizier.cds.unistra.fr/viz-bin/VizieR-3?-source=I/345/gaia2
catalog_name = "I/350/gaiaedr3"   # https://vizier.cds.unistra.fr/viz-bin/VizieR-3?-source=I/350/gaiaedr3

radius_arcsec = 30

# catalog_name, radius_arcsec = "B/vsx/vsx", 1200

magnitude_limit = 20
ra, dec = 263.59892769, 74.47212807

import warnings
with warnings.catch_warnings():
    # workaround https://github.com/astropy/astroquery/issues/2352
    warnings.filterwarnings("ignore", category=astropy.units.UnitsWarning)
#     warnings.filterwarnings("ignore", message=".*Unit 'e' not supported by the VOUnit standard.*")
#     warnings.simplefilter("ignore")
    result = search_nearby(ra, dec, catalog_name=catalog_name, radius_arcsec=radius_arcsec, magnitude_limit=magnitude_limit)
print(f"From {catalog_name}:")
result


From I/350/gaiaedr3:


RA_ICRS,e_RA_ICRS,DE_ICRS,e_DE_ICRS,Source,Plx,e_Plx,PM,pmRA,e_pmRA,pmDE,e_pmDE,RUWE,FG,e_FG,Gmag,e_Gmag,FBP,e_FBP,BPmag,e_BPmag,FRP,e_FRP,RPmag,e_RPmag,BP-RP,RVDR2,e_RVDR2,Tefftemp,loggtemp,PS1,SDSSDR13,SkyMapper2,URAT1,GmagCorr,e_GmagCorr,FGCorr,RAJ2000,DEJ2000,epsi,sepsi
deg,mas,deg,mas,,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,e-/s,e-/s,mag,mag,e-/s,e-/s,mag,mag,e-/s,e-/s,mag,mag,mag,km / s,km / s,K,[cm/s2],,,,,mag,mag,mag,deg,deg,mas,
float64,float64,float64,float64,int64,float64,float32,float64,float64,float32,float64,float32,float64,float32,float32,float64,float64,float32,float32,float64,float64,float32,float32,float64,float64,float64,float64,float32,float32,float32,int64,int64,int32,int32,float64,float64,float32,float64,float64,float64,float32
263.57197172331,0.4007,74.47405770248,0.3897,1655666290100696320,0.4514,0.3970,2.530,-2.360,0.461,-0.912,0.453,0.932,203.6,0.9553,19.915178,0.005791,85.91,7.734,20.503456,0.097778,109.8,8.025,19.645914,0.079406,0.857542,--,--,--,--,197362635720789659,--,--,--,19.915178,0.005791,203.6,263.57201091240,74.47406175515,0.000,0
263.60987497374,0.2231,74.47967608122,0.2438,1655666290100924672,0.7793,0.2410,6.726,-3.507,0.258,5.740,0.278,1.065,432.1,1.102,19.098475,0.003905,138.8,10.9,19.982477,0.085280,470.9,8.331,18.065605,0.019578,1.916872,--,--,--,--,197372636098846406,--,--,823077878,19.098475,0.003905,432.1,263.60993321655,74.47965057040,0.725,1.2
263.59315820613,0.2527,74.47200326289,0.2768,1655666294395083264,1.4577,0.2727,14.053,-0.340,0.281,-14.049,0.334,1.116,365.8,1.434,19.279320,0.005070,157.6,17.91,19.844767,0.123434,502.9,19.32,17.994108,0.041884,1.850658,--,--,--,--,197362635932607221,--,--,--,19.273023,0.005049,367.9,263.59316385040,74.47206570350,1.033,2.1
263.59890609490,0.0124,74.47216418802,0.0155,1655666294396195200,1.1199,0.0141,21.763,3.595,0.014,-21.464,0.018,1.144,1.795e+05,49.88,12.552287,0.002772,1.013e+05,57.83,12.824336,0.002858,1.138e+05,52.36,12.107111,0.003812,0.717225,-25.37,6.08,5500.0,3.5,--,--,--,823077874,12.552287,0.002772,1.795e+05,263.59884641720,74.47225958509,0.093,9.1
263.60223514318,0.0170,74.47989152499,0.0207,1655666294396195968,0.2521,0.0190,15.065,-4.886,0.020,14.251,0.024,0.970,2.813e+04,6.321,14.564581,0.002766,1.408e+04,20.57,14.966750,0.003209,2.02e+04,19.92,13.984493,0.003928,0.982257,--,--,--,--,197372636022496651,--,--,823077876,14.564581,0.002766,2.813e+04,263.60231629273,74.47982818744,0.000,0
263.60127387359,0.4886,74.47693496878,0.4630,1655666294396731520,-0.1614,0.5317,4.457,-4.437,0.511,0.417,0.603,0.986,161.9,0.8168,20.163914,0.006130,58.69,5.799,20.917192,0.107314,145.4,8.645,19.341621,0.064674,1.575571,--,--,--,--,197372636013273126,--,--,--,20.161801,0.006120,162.3,263.60134756450,74.47693311483,0.000,0


In [29]:
def search_nearby(ra, dec, catalog_name="I/345/gaia2", radius_arcsec=30):    
    c1 = SkyCoord(ra, dec, frame="icrs", unit="deg")
    columns = ["RAJ2000", "DEJ2000", "Source", "Gmag", "FG"]

    result = Vizier(columns=columns).query_region(
        c1,
        catalog=[catalog_name],
        radius=Angle(radius_arcsec, "arcsec"),
    )
    return result[catalog_name]

ra, dec = 263.59892769, 74.47212807
result = search_nearby(ra, dec)
result


RAJ2000,DEJ2000,Source,Gmag,FG
deg,deg,,mag,e-/s
float64,float64,int64,float64,float32
263.60993198300,74.47965554850,1655666290100924672,19.1363,417.6
263.59316051900,74.47206318180,1655666294395083264,19.2750,367.6
263.60231827500,74.47982857640,1655666294396195968,14.5761,2.786e+04
263.59884868200,74.47225880000,1655666294396195200,12.5566,1.789e+05
263.57202750900,74.47406795860,1655666290100696320,19.9597,195.6
263.60135982300,74.47693106390,1655666290100697856,20.2177,154.3


In [95]:
# from astropy.units import cds
# result["FG"].unit == cds.e / u.second
# result["FG"].unit == u.electron / u.second

result["FG"][0], result["FG"].to(u.electron / u.second)[0]


ValueError: The unit 'e-/s' is unrecognized.  It can not be converted to other units.

In [48]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia

Gaia.ROW_LIMIT = 50  # Ensure the default row limit.
ra, dec = 263.59892769, 74.47212807
coord = SkyCoord(ra=ra, dec=dec, unit=(u.degree, u.degree), frame='icrs')
radius = u.Quantity(30, u.arcsec)
job = Gaia.cone_search(coord, radius)
res = job.get_results()
res

solution_id,DESIGNATION,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,astrometric_weight_al,astrometric_pseudo_colour,astrometric_pseudo_colour_error,mean_varpi_factor_al,astrometric_matched_observations,visibility_periods_used,astrometric_sigma5d_max,frame_rotator_object_type,matched_observations,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_nb_transits,rv_template_teff,rv_template_logg,rv_template_fe_h,phot_variable_flag,l,b,ecl_lon,ecl_lat,priam_flags,teff_val,teff_percentile_lower,teff_percentile_upper,a_g_val,a_g_percentile_lower,a_g_percentile_upper,e_bp_min_rp_val,e_bp_min_rp_percentile_lower,e_bp_min_rp_percentile_upper,flame_flags,radius_val,radius_percentile_lower,radius_percentile_upper,lum_val,lum_percentile_lower,lum_percentile_upper,datalink_url,dist
,,,,yr,deg,mas,deg,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / mas2,1 / um,1 / um,,,,mas,,,,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,,mag,mag,mag,km / s,km / s,,K,log(cm.s**-2),dex,,deg,deg,deg,deg,,K,K,K,mag,mag,mag,mag,mag,mag,,solRad,solRad,solRad,solLum,solLum,solLum,,
int64,object,int64,int64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,float32,float32,float64,float64,int16,bool,float32,float64,float64,float32,int16,int16,float32,int32,int16,bool,int32,float64,float64,float32,float32,int32,float64,float64,float32,float32,int32,float64,float64,float32,float32,float32,int16,float32,float32,float32,float64,float64,int32,float32,float32,float32,object,float64,float64,float64,float64,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32,float32,float32,float32,object,float64
1635721458409799680,Gaia DR2 1655666294396195200,1655666294396195200,1358916289,2015.5,263.59890403460577,0.03170163894732282,74.47216720646641,0.04892681686518623,1.108937161662669,0.04655582626681095,23.819513,3.4416501411560043,0.04967232464463041,-21.273350405440084,0.07429961516390249,-0.2610099,-0.56073153,0.09585507,0.10948105,0.6266922,-0.32973415,-0.62051433,-0.44896287,-0.39297086,0.12271285,167,167,166,1,4.6090813,257.76434,0.0,0.0,31,True,247.27481,1.6407132332405652,0.008580896601978938,-0.11661293,19,14,0.054497305,0,22,False,193,178932.07109762123,45.150793732790945,3962.9883,12.556645,21,101483.92004431318,208.0702882847174,487.73865,12.835395,21,114246.63390573513,114.7918250967366,995.2506,12.1173115,1.2056562,0,0.7180834,0.27874947,0.43933392,-25.372872488897567,6.081422837677936,5,5500.0,3.5,-1.5,NOT_AVAILABLE,105.74822462216561,31.19368643395216,102.103825911288,81.81684407487553,100001,6141.0,5891.0,6440.0,--,--,--,--,--,--,200111,2.1014493,1.9108446,2.2835948,5.6583385,5.325697,5.99098,https://gea.esac.esa.int/data-server/datalink/links?ID=Gaia+DR2+1655666294396195200,3.429458889269471e-05
1635721458409799680,Gaia DR2 1655666294395083264,1655666294395083264,1169295133,2015.5,263.5931588177774,0.35105221587843655,74.4720050431908,0.512853972479

In [94]:
# res["phot_g_mean_flux"].unit == u.electron / u.second

res["phot_g_mean_flux"].to(u.electron/u.second)

UnitConversionError: 'electron / s' and 'electron / s' are not convertible

In [93]:
from astropy.table import Table, QTable
from astropy.io import fits

def try_tess_fits(): 
    fitspath = "data/mastDownload/TESS/tess2022027120115-s0048-0000001718047913-0219-s/tess2022027120115-s0048-0000001718047913-0219-s_lc.fits"
    with fits.open(fitspath) as hduList:
            num_hdus = len(hduList)
    # for i in range(1, num_hdus):
    #     table = Table.read(hduList[i], format='fits')    
            table = Table.read(hduList[1], format='fits')
    return table["SAP_FLUX"].unit

flux_u = try_tess_fits()
flux_u.to(u.electron/u.second)

ValueError: The unit 'e-/s' is unrecognized.  It can not be converted to other units.

In [1]:
import astropy
import astroquery

print("Astroquery: ", astroquery.__version__)
print("Astropy: ", astropy.__version__)

Astroquery:  0.4.2
Astropy:  4.2.1


### Conversion between heliocentric times and barycentric time

In [ ]:
# From https://gist.github.com/StuartLittlefair/4ab7bb8cf21862e250be8cb25f72bb7a

from astropy.coordinates import SkyCoord, EarthLocation
from astropy import units as u
from astropy.time import Time

def helio_to_bary(coords, hjd, obs_name):
    helio = Time(hjd, scale='utc', format='jd')
    obs = EarthLocation.of_site(obs_name)
    star = SkyCoord(coords, unit=(u.hour, u.deg)) 
    ltt = helio.light_travel_time(star, 'heliocentric', location=obs)
    guess = helio - ltt
    # if we assume guess is correct - how far is heliocentric time away from true value?
    delta = (guess + guess.light_travel_time(star, 'heliocentric', obs)).jd  - helio.jd
    # apply this correction
    guess -= delta * u.d

    ltt = guess.light_travel_time(star, 'barycentric', obs)
    return guess.tdb + ltt


def bary_to_helio(coords, bjd, obs_name):
    bary = Time(bjd, scale='tdb', format='jd')
    obs = EarthLocation.of_site(obs_name)
    star = SkyCoord(coords, unit=(u.hour, u.deg))
    ltt = bary.light_travel_time(star, 'barycentric', location=obs) 
    guess = bary - ltt
    delta = (guess + guess.light_travel_time(star, 'barycentric', obs)).jd  - bary.jd
    guess -= delta * u.d
    
    ltt = guess.light_travel    
    

### Time format conversion

In [2]:
from astropy.time import Time
import lightkurve as lk  # for BTJD format

# t1 = Time("2019-05-01", format="iso", scale="utc")
# t1 = Time("58604.4", format="mjd", scale="tdb") # BMJD, from the paper
# t1 = Time("2459307.239419309", format="jd", scale="tdb")  # BJD
t1 = Time("2613.135", format="btjd")  # 2633, 1875.55
t1 = Time("2611.240", format="btjd")  # 2633, 1875.55
# t1 = Time(7264.23209 + 2450000, format="jd", scale="utc")  # HJD

print("Source")
print(t1.to_datetime())


t2 = t1.copy()
# t2.format = "btjd" 
t2.format="jd"
# potential issue issue: scale is still utc, can't convert to Barycentric

print("Converted:")
print(t2.to_datetime())
t2

Source
2022-01-31 17:45:36
Converted:
2022-01-31 17:45:36


<Time object: scale='tdb' format='jd' value=2459611.24>

### Time zone conversion

In [37]:
# adhoc time conversion helper

import pytz
from astropy.time import Time
import lightkurve as lk  # for BTJD format

# t1 = Time("2019-05-01", format="iso", scale="utc")
# t1 = Time("58604.4", format="mjd", scale="tdb") # BMJD, from the paper
t1 = Time(7264.23209 + 2450000, format="jd", scale="utc")  # HJD
# t1 = Time("2459307.239419309", format="jd", scale="tdb")  # BJD
# t1 = Time("2032.093", format="btjd")  # epoch for TIC 284361752, TOI 2294.01 (long period)
dt = t1.to_datetime()
print(dt)
print(pytz.timezone('US/Pacific').fromutc(dt))


2015-08-29 17:34:12.576005
2015-08-29 10:34:12.576005-07:00
